In [3]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [1]:
!pip install schedule

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install jupyter_scheduler

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/103.4 kB ? eta -:--:--
     -------------------------------------- 103.4/103.4 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/518.4 kB ? eta -:--:--
   ------------- -------------------------- 174.1/518.4 kB 3.6 MB/s eta 0:00:01
   -------------------------------- ------- 419.8/518.4 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 518.4/518.4 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/163.8 kB ? eta -:--:--
   ------------------------------------- -- 153.6/163.8 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 163.8/163.8 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/502.3 kB ? eta -:--:--
   ------------- -------------------------- 174.1/502.3 kB 5.3 MB/s eta 0:00:01
   ---------------------------------- ----- 430.1/502.3 kB 5.4 MB/s eta 0:00

In [3]:
ChromeDriverManager().install()

'C:\\Users\\jw\\.wdm\\drivers\\chromedriver\\win64\\123.0.6312.122\\chromedriver-win32/chromedriver.exe'

In [4]:
#라이브러리 
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
import pandas as pd
from datetime import datetime, timedelta
import xlsxwriter 
import schedule

1번째 코드: 전체 데이터를 한번에 올리는 코드 

In [20]:
browser = webdriver.Chrome()
url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
browser.get(url)
browser.find_element(By.CLASS_NAME, "ui-datepicker-trigger").click() # 데이트피커 클릭
time.sleep(1)
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/div/a[1]").click() # 이전달 클릭
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/table/tbody/tr[4]/td[7]/a").click()
all_data = pd.DataFrame()  # 모든 데이터를 저장할 데이터프레임 초기화
dt = datetime.today() - timedelta(days=1)
dt_str = dt.strftime("%Y.%m.%d")
while True:
    time.sleep(1)
    date = browser.find_element(By.CLASS_NAME, "date").text
    table = browser.find_element(By.CLASS_NAME, "tData").text
    lines = table.strip().split('\n')
    rows = [line.split() for line in lines]
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df = df.drop(columns=['게임차', '연속', '홈', '방문'])
    df['Date'] = date
    cols = df.columns.tolist()
    cols = ['Date'] + [col for col in cols if col != 'Date']
    df = df[cols]
    all_data = pd.concat([all_data, df], ignore_index=True)  # 기존 데이터프레임과 새로운 데이터프레임을 합침
    all_data.to_csv('basketball_practice.csv', index=False, encoding='utf-8-sig')  # 파일 저장 
    next_button = browser.find_element(By.CLASS_NAME, "date_next")  
    next_button.click()
    if browser.find_element(By.CLASS_NAME, "date").text==dt_str:
        break
 # 3월23일 클릭

In [8]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


2번째 코드 : exel_writer로 나누어서 저장하는 코드 

In [6]:
browser = webdriver.Chrome()
url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
browser.get(url)
browser.find_element(By.CLASS_NAME, "ui-datepicker-trigger").click() # 데이트피커 클릭
time.sleep(1)
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/div/a[1]").click() # 이전달 클릭
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/table/tbody/tr[4]/td[7]/a").click()
dt = datetime.today() - timedelta(days=1)
dt_str = dt.strftime("%Y.%m.%d")
excel_writer = pd.ExcelWriter('basketball_data.xlsx', engine='xlsxwriter')
while True:
    time.sleep(1)
    table = browser.find_element(By.CLASS_NAME, "tData").text
    lines = table.strip().split('\n')
    rows = [line.split() for line in lines]
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df = df.drop(columns=['게임차', '연속', '홈', '방문'])
    sheet_name = browser.find_element(By.CLASS_NAME, "date").text
    df.to_excel(excel_writer, index=False, sheet_name=sheet_name)
    next_button = browser.find_element(By.CLASS_NAME, "date_next")  
    next_button.click()
    if browser.find_element(By.CLASS_NAME, "date").text==dt_str:
        break
excel_writer.close()


마지막 최신날짜까지 찾아오는데까지 업데이트 완료

In [10]:
browser = webdriver.Chrome()
url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
browser.get(url)
recent_day = browser.find_element(By.CLASS_NAME, "date").text
browser.find_element(By.CLASS_NAME, "ui-datepicker-trigger").click() # 데이트피커 클릭
time.sleep(1)
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/div/a[1]").click() # 이전달 클릭
browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/table/tbody/tr[4]/td[7]/a").click()
excel_writer = pd.ExcelWriter('baseball_data.xlsx', engine='xlsxwriter')
while True:
    time.sleep(1)
    table = browser.find_element(By.CLASS_NAME, "tData").text
    lines = table.strip().split('\n')
    rows = [line.split() for line in lines]
    df = pd.DataFrame(rows[1:], columns=rows[0])
    df = df.drop(columns=['게임차', '연속', '홈', '방문'])
    sheet_name = browser.find_element(By.CLASS_NAME, "date").text
    df.to_excel(excel_writer, index=False, sheet_name=sheet_name)
    next_button = browser.find_element(By.CLASS_NAME, "date_next")  
    next_button.click()
    date = browser.find_element(By.CLASS_NAME, "date").text
    if date == recent_day:
        print("데이터 수집 완료")
        break   
excel_writer.close()

Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


데이터 수집 완료


#에러사항: value값에 공백이 있어서 안 찾아지고, 반복문 없앨 경우 None이나 ''로 설정되어있어서 이것으로 멈출 수 없음

In [51]:
browser = webdriver.Chrome()
url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
browser.get(url)
input_element = browser.find_element(By.ID, 'cphContents_cphContents_cphContents_txtCanlendar')
value = input_element.get_property('value')
value

'20240410  '

In [5]:
def python_project():
    #라이브러리 
    browser = webdriver.Chrome()
    url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
    browser.get(url)
    recent_day = browser.find_element(By.CLASS_NAME, "date").text
    browser.find_element(By.CLASS_NAME, "ui-datepicker-trigger").click() # 데이트피커 클릭
    time.sleep(1)
    browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/div/a[1]").click() # 이전달 클릭
    browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/table/tbody/tr[4]/td[7]/a").click()
    excel_writer = pd.ExcelWriter('baseball_data.xlsx', engine='xlsxwriter')
    while True:
        time.sleep(1)
        table = browser.find_element(By.CLASS_NAME, "tData").text
        lines = table.strip().split('\n')
        rows = [line.split() for line in lines]
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df = df.drop(columns=['게임차', '연속', '홈', '방문'])
        sheet_name = browser.find_element(By.CLASS_NAME, "date").text
        df.to_excel(excel_writer, index=False, sheet_name=sheet_name)
        next_button = browser.find_element(By.CLASS_NAME, "date_next")  
        next_button.click()
        date = browser.find_element(By.CLASS_NAME, "date").text
        if date == recent_day:
            print("데이터 수집 완료")
            break   
    excel_writer.close()
python_project()
schedule.every().day.at("08:00").do(python_project)
while True:
    schedule.run_pending()
    time.sleep(1)

데이터 수집 완료


KeyboardInterrupt: 

In [5]:
python_project()

데이터 수집 완료


In [11]:
browser = webdriver.Chrome()
url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
browser.get(url)
time.sleep(1)
all_data = pd.DataFrame()  # 모든 데이터를 저장할 데이터프레임 초기화
recent_day = browser.find_element(By.CLASS_NAME, "date").text
time.sleep(1)
table = browser.find_element(By.CLASS_NAME, "tData").text
lines = table.strip().split('\n')
rows = [line.split() for line in lines]
df = pd.DataFrame(rows[1:], columns=rows[0])
df = df.drop(columns=['게임차', '연속', '홈', '방문'])
all_data = pd.concat([all_data, df], ignore_index=True)  # 기존 데이터프레임과 새로운 데이터프레임을 합침
all_data.to_csv('baseball_practice.csv', index=False, encoding='utf-8-sig')  # 파일 저장

In [ ]:
crawling_homework/project3.ipynb

In [ ]:
def python_project():
    #라이브러리 
    browser = webdriver.Chrome()
    url = 'https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx'
    browser.get(url)
    recent_day = browser.find_element(By.CLASS_NAME, "date").text
    browser.find_element(By.CLASS_NAME, "ui-datepicker-trigger").click() # 데이트피커 클릭
    time.sleep(1)
    browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/div/a[1]").click() # 이전달 클릭
    browser.find_element(By.XPATH, "//*[@id=\"ui-datepicker-div\"]/table/tbody/tr[4]/td[7]/a").click()
    excel_writer = pd.ExcelWriter('baseball_data.xlsx', engine='xlsxwriter')
    while True:
        time.sleep(1)
        table = browser.find_element(By.CLASS_NAME, "tData").text
        lines = table.strip().split('\n')
        rows = [line.split() for line in lines]
        df = pd.DataFrame(rows[1:], columns=rows[0])
        df = df.drop(columns=['게임차', '연속', '홈', '방문'])
        sheet_name = browser.find_element(By.CLASS_NAME, "date").text
        df.to_excel(excel_writer, index=False, sheet_name=sheet_name)
        next_button = browser.find_element(By.CLASS_NAME, "date_next")  
        next_button.click()
        date = browser.find_element(By.CLASS_NAME, "date").text
        if date == recent_day:
            print("데이터 수집 완료")
            break   
    excel_writer.close()
python_project()
schedule.every().day.at("08:00").do(python_project)
while True:
    schedule.run_pending()
    time.sleep(1)